<a href="https://colab.research.google.com/github/binliu0630/Deep_Learning/blob/master/Multilabel_Toxic_Comments_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Data

In [2]:
!pip install kaggle

In [3]:
!ls

sample_data


In [4]:
!rm -r ~/.kaggle # remove .kaggle

rm: cannot remove '/root/.kaggle': No such file or directory


In [0]:
!mkdir .kaggle

In [13]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [0]:
#!touch ~/.kaggle/kaggle.json

In [0]:
import json
token = {"username":"binliu0630","key":"8fbf745edbb9799f7f8f45f516701290"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [16]:
!ls ~/.kaggle

kaggle.json


In [17]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [19]:
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge -p /content

 99% 26.0M/26.3M [00:00<00:00, 36.7MB/s]
100% 26.3M/26.3M [00:00<00:00, 59.5MB/s]
 38% 9.00M/23.4M [00:00<00:01, 13.1MB/s]
100% 23.4M/23.4M [00:00<00:00, 30.1MB/s]
  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 92.7MB/s]
  0% 0.00/1.46M [00:00<?, ?B/s]
100% 1.46M/1.46M [00:00<00:00, 204MB/s]


In [20]:
!unzip \*.zip

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   

Archive:  test_labels.csv.zip
  inflating: test_labels.csv         

Archive:  train.csv.zip
  inflating: train.csv               

Archive:  test.csv.zip
  inflating: test.csv                

4 archives were successfully processed.


In [21]:
!ls

sample_data		   test.csv	    test_labels.csv.zip
sample_submission.csv	   test.csv.zip     train.csv
sample_submission.csv.zip  test_labels.csv  train.csv.zip


In [22]:
import pandas as pd

df = pd.read_csv('train.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


# Get Started with Transformers

In [23]:
!pip install transformers

     |████████████████████████████████| 665kB 9.1MB/s 
     |████████████████████████████████| 3.8MB 25.0MB/s 
     |████████████████████████████████| 1.1MB 55.4MB/s 
     |████████████████████████████████| 890kB 51.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b24b45e4f2df43d7c727febae68f410b7f239233e3f8da6d0387df6467dbc8cf
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf

In [25]:
tf.__version__

'2.2.0'

### Data Pipeline

In [0]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']

In [0]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [0]:
df_test_labels = pd.read_csv('test_labels.csv')
df_test_labels = df_test_labels.set_index('id')

In [29]:
df_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [0]:
from transformers import BertTokenizer

In [31]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [0]:
MAX_LEN = 64

In [0]:
# only take first 5000 as example
token = tokenizer.batch_encode_plus(df_train['comment_text'].values, max_length=64, pad_to_max_length=True)

In [34]:
token.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [0]:
input_ids = token['input_ids']
attention_mask = token['attention_mask']

In [0]:
from sklearn.model_selection import train_test_split
labels = df_train[label_cols].values

In [0]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels,random_state=1234, test_size=0.1)

In [0]:
train_masks, validation_masks, _, _ = train_test_split(attention_mask, labels, random_state=1234, test_size=0.1)

In [39]:
train_size = len(train_inputs)
print(f'train size: {train_size}')
validation_size = len(validation_inputs)
print(f'validation size: {validation_size}')

train size: 143613
validation size: 15958


In [0]:
BATCH_SIZE = 32
EPOCHS = 1

def create_dataset(data_tuple, epochs=1, batch_size=32, buffer_size=10000, train=True):
  dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
  if train:
    dataset = dataset.shuffle(buffer_size=buffer_size)
  dataset = dataset.repeat(epochs)
  dataset = dataset.batch(batch_size)
  if train:
    dataset = dataset.prefetch(1)
  return dataset 

In [0]:
train_dataset = create_dataset((train_inputs, train_masks, train_labels), 
                               epochs=EPOCHS, batch_size=BATCH_SIZE)

In [0]:
validation_dataset = create_dataset((validation_inputs, validation_masks, validation_labels),
                                    epochs=EPOCHS, batch_size=BATCH_SIZE)

#### Pytorch BERT MODEL

In [0]:
from transformers import BertModel




#### TF BERT MODEL

In [0]:
from transformers import TFBertModel
from tensorflow.keras.layers import Dense, Flatten

class BertMultiLabelClassifier(tf.keras.Model):
  def __init__(self, bert: TFBertModel, num_classes: int):
    super().__init__()
    self.bert = bert
    self.classifier = Dense(num_classes, activation='sigmoid')
  @tf.function
  def call(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
    outputs = self.bert(input_ids, 
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids,
                       position_ids=position_ids,
                       head_mask=head_mask)
    cls_output = outputs[1]
    cls_output = self.classifier(cls_output)
    

    return cls_output
model = BertMultiLabelClassifier(TFBertModel.from_pretrained('bert-base-uncased'), len(label_cols))


In [0]:
import time
from transformers import create_optimizer

steps_per_epoch = train_size // BATCH_SIZE
validation_steps = validation_size // BATCH_SIZE

# | Loss Function
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=False)
train_loss = tf.keras.metrics.Mean(name='train_loss')
validation_loss = tf.keras.metrics.Mean(name='test_loss')

# | Optimizer (with 1-cycle-policy)
warmup_steps = steps_per_epoch // 3
total_steps = steps_per_epoch * EPOCHS - warmup_steps
optimizer = create_optimizer(init_lr=2e-5, num_train_steps=total_steps, num_warmup_steps=warmup_steps)

# | Metrics
train_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]
validation_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]

@tf.function
def train_step(model, token_ids, masks, labels):
    labels = tf.dtypes.cast(labels, tf.float32)
    print(labels.shape)

    with tf.GradientTape() as tape:
        predictions = model(token_ids, attention_mask=masks)
        print(predictions.shape)
        loss = loss_object(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)

    for i, auc in enumerate(train_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])
    

In [73]:
for i, (token_ids, masks, labels) in enumerate(tqdm(train_dataset, total=steps_per_epoch)):
    #print((token_ids, masks, labels))
    train_step(model,token_ids, masks, labels )
    break

(32, 6)
(32, 6)
(32, 6)
(32, 6)


In [74]:
import time
from transformers import create_optimizer

steps_per_epoch = train_size // BATCH_SIZE
validation_steps = validation_size // BATCH_SIZE

# | Loss Function
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=False)
train_loss = tf.keras.metrics.Mean(name='train_loss')
validation_loss = tf.keras.metrics.Mean(name='test_loss')

# | Optimizer (with 1-cycle-policy)
warmup_steps = steps_per_epoch // 3
total_steps = steps_per_epoch * EPOCHS - warmup_steps
optimizer = create_optimizer(init_lr=2e-5, num_train_steps=total_steps, num_warmup_steps=warmup_steps)

# | Metrics
train_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]
validation_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]

@tf.function
def train_step(model, token_ids, masks, labels):
    labels = tf.dtypes.cast(labels, tf.float32)

    with tf.GradientTape() as tape:
        predictions = model(token_ids, attention_mask=masks)
        loss = loss_object(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)

    for i, auc in enumerate(train_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])
        
@tf.function
def validation_step(model, token_ids, masks, labels):
    labels = tf.dtypes.cast(labels, tf.float32)

    predictions = model(token_ids, attention_mask=masks, training=False)
    v_loss = loss_object(labels, predictions)

    validation_loss(v_loss)
    for i, auc in enumerate(validation_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])
                                             
def train(model, train_dataset, val_dataset, train_steps_per_epoch, val_steps_per_epoch, epochs):
    for epoch in range(epochs):
        print('=' * 50, f"EPOCH {epoch}", '=' * 50)

        start = time.time()

        for i, (token_ids, masks, labels) in enumerate(tqdm(train_dataset, total=train_steps_per_epoch)):
            train_step(model, token_ids, masks, labels)
            if i % 1000 == 0:
                print(f'\nTrain Step: {i}, Loss: {train_loss.result()}')
                for i, label_name in enumerate(label_cols):
                    print(f"{label_name} roc_auc {train_auc_metrics[i].result()}")
                    train_auc_metrics[i].reset_states()
        
        for i, (token_ids, masks, labels) in enumerate(tqdm(val_dataset, total=val_steps_per_epoch)):
            validation_step(model, token_ids, masks, labels)

        print(f'\nEpoch {epoch+1}, Validation Loss: {validation_loss.result()}, Time: {time.time()-start}\n')

        for i, label_name in enumerate(label_cols):
            print(f"{label_name} roc_auc {validation_auc_metrics[i].result()}")
            validation_auc_metrics[i].reset_states()

        print('\n')

        
train(model, train_dataset, validation_dataset, train_steps_per_epoch=steps_per_epoch, val_steps_per_epoch=validation_steps, epochs=EPOCHS)

================================================== EPOCH 0 ==================================================



Train Step: 0, Loss: 0.759718656539917
toxic roc_auc 0.3169642686843872
severe_toxic roc_auc 0.0
obscene roc_auc 0.3583333492279053
threat roc_auc 0.0
insult roc_auc 0.0
identity_hate roc_auc 0.0

Train Step: 1000, Loss: 0.15975356101989746
toxic roc_auc 0.8934990763664246
severe_toxic roc_auc 0.8807864785194397
obscene roc_auc 0.9029343128204346
threat roc_auc 0.68030846118927
insult roc_auc 0.878491997718811
identity_hate roc_auc 0.7899352312088013

Train Step: 2000, Loss: 0.10363499075174332
toxic roc_auc 0.9754371643066406
severe_toxic roc_auc 0.9836423993110657
obscene roc_auc 0.9807364344596863
threat roc_auc 0.9467393159866333
insult roc_auc 0.9777823686599731
identity_hate roc_auc 0.9625445008277893

Train Step: 3000, Loss: 0.08319544792175293
toxic roc_auc 0.9808622598648071
severe_toxic roc_auc 0.9864641427993774
obscene roc_auc 0.9832154512405396
threat roc_auc 0.9414742588996887
insult roc_auc 0.9822421073913574
identity_hate roc_auc 0.9782419800758362

Train Step: 4000, L



Epoch 1, Validation Loss: 0.04019291698932648, Time: 1953.9268221855164

toxic roc_auc 0.9802592992782593
severe_toxic roc_auc 0.9918339848518372
obscene roc_auc 0.9836854934692383
threat roc_auc 0.9919807314872742
insult roc_auc 0.9797335267066956
identity_hate roc_auc 0.9775397181510925


